In [3]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import datetime
import json
import time
import requests

### 1) Scraping the general information for all articles
- ticker
- category
- release_date
- provider
- url
- article_id


In [12]:
links = { "AAPL" : "https://www.investing.com/equities/apple-computer-inc",
          "MSFT" : "https://www.investing.com/equities/microsoft-corp",
          "GOOGL" : "https://www.investing.com/equities/google-inc",
          "AMZN" : "https://www.investing.com/equities/amazon-com-inc",
          "FB" : "https://www.investing.com/equities/facebook-inc",
          "TSLA" : "https://www.investing.com/equities/tesla-motors",
          "BRK-A" : "https://www.investing.com/equities/berkshire-hathaway",
          "TSM" : "https://www.investing.com/equities/taiwan-semicond.manufacturing-co",
          "NVDA" : "https://www.investing.com/equities/nvidia-corp",
          "JPM" : "https://www.investing.com/equities/jp-morgan-chase",
          "GME" : "https://www.investing.com/equities/gamestop-corp",
          #"V" : "https://www.investing.com/equities/visa-inc",
          #"BABA" : "https://www.investing.com/equities/alibaba",
          #"JNJ" : "https://www.investing.com/equities/johnson-johnson",
          #"WMT" : "https://www.investing.com/equities/wal-mart-stores",
          #"UNH" : "https://www.investing.com/equities/united-health-group",
          #"BAC" : "https://www.investing.com/equities/bank-of-america",
          #"HD" : "https://www.investing.com/equities/home-depot",
          #"MA" : "https://www.investing.com/equities/mastercard-cl-a",
          #"PG" : "https://www.investing.com/equities/procter-gamble",
          #"DIS" : "https://www.investing.com/equities/disney"
         }


news = pd.DataFrame([])
n_req = 1
latest_date = datetime.datetime(2022,5,1)


for ticker, l in links.items():
  
  for category in ['news', 'opinion']:
    n = 1
    prev_id = 0
    cont = True
    while cont:
      
      if n == 1: 
        link = l + "-" + category
      else:
        link = l + "-" + category + "/" + str(n)

      r = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
      response = urlopen(r).read()
      soup = BeautifulSoup(response, "html.parser")
      if category == 'news':
        table = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ["mediumTitle1"])
      else:
        table = soup.select('div.mediumTitle1.analysisImg')[0]

      table = table.find_all('article')

      result = []
      base = {}

      for a in table:
        article_id = a['data-id']
        provider = a.find(class_ = "articleDetails").find('span').text.replace('By ', '')
        date = a.find(class_ = "date").text[3:]
        if 'ago' in date:
          date = datetime.date.today()
        else:
          date = datetime.datetime.strptime(date, '%b %d, %Y').date()


        href = a.find(class_ = "title")["href"]
        if href.startswith('/'):
          href = "https://www.investing.com" + href
        title = a.find(class_ = "title")["title"]

        new = pd.DataFrame({
                            'ticker':[ticker],
                            'title':[title],
                            'category':[category],
                            'release_date':[date],
                            'provider':[provider],
                            'url':[href],
                            'article_id':[article_id],
                            'content':[''],
                            'release_datetime':['']
                            })
        news = pd.concat([news, new])


      print(str(n_req) + ' - Ticker ' + ticker + ' - ' + category + ' - Page ' + str(n) + ' - Last date ' + str(date))
      n += 1
      n_req += 1

      if (date >= latest_date.date()) and (prev_id != article_id): 
        cont = True
        prev_id = article_id

      else:
        cont = False
    



1 - Ticker AAPL - news - Page 1 - Last date 2022-05-03
2 - Ticker AAPL - news - Page 2 - Last date 2022-05-02
3 - Ticker AAPL - news - Page 3 - Last date 2022-05-02
4 - Ticker AAPL - news - Page 4 - Last date 2022-04-29
5 - Ticker AAPL - news - Page 5 - Last date 2022-04-29
6 - Ticker AAPL - news - Page 6 - Last date 2022-04-28
7 - Ticker AAPL - news - Page 7 - Last date 2022-04-27
8 - Ticker AAPL - news - Page 8 - Last date 2022-04-27
9 - Ticker AAPL - news - Page 9 - Last date 2022-04-26
10 - Ticker AAPL - news - Page 10 - Last date 2022-04-25
11 - Ticker AAPL - news - Page 11 - Last date 2022-04-23
12 - Ticker AAPL - news - Page 12 - Last date 2022-04-22
13 - Ticker AAPL - news - Page 13 - Last date 2022-04-20
14 - Ticker AAPL - news - Page 14 - Last date 2022-04-19
15 - Ticker AAPL - news - Page 15 - Last date 2022-04-15
16 - Ticker AAPL - news - Page 16 - Last date 2022-04-13
17 - Ticker AAPL - news - Page 17 - Last date 2022-04-12
18 - Ticker AAPL - news - Page 18 - Last date 202

KeyboardInterrupt: ignored

In [21]:
timestr = time.strftime("%Y%m%d-%H%M%S")
name = "News/news_" + timestr + ".csv"
print(name)
news.to_csv(name, index=False)

News/news_20220505-094559.csv


### 2) Requesting the url of each article
- article content
- release date

In [27]:
max_req = 20000
j=0
for i, (link, content) in enumerate(zip(df['url'], df['content'])):
  if j <= max_req:
    if pd.isnull(content):
      j+=1
      if (j % 500) == 0:
        print('--- saving ---')
        name = "News/news_scraped.csv"
        df.to_csv(name, index=False)
        
      try:
        r = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        response = urlopen(r).read()
        soup = BeautifulSoup(response, "html.parser")
        table = soup.find(class_='articlePage').find_all('p')
        content = ' '.join([t.text for t in table])

        if content == '':
          table = soup.find(class_='articlePage').find_all('div', class_='')
          content = ' '.join([t.text for t in table])
          
          if content == '':
            table = soup.find(class_='articlePage').find_all('li')
            content = ' '.join([t.text for t in table])

        df.loc[i, 'content'] = content

        release_datetime = soup.find(class_ = 'contentSectionDetails').span.text
        df.loc[i, 'release_datetime'] = release_datetime

        print('request number ' + str(j) + ' - ' + df.loc[i, 'provider'] + ' - SUCCESS - ' + df.loc[i, 'url'] )

      except:
        table = soup.find_all('p')
        content = ' '.join([t.text for t in table])
        df.loc[i, 'content'] = content
        try:
          release_datetime = soup.time.text
          df.loc[i, 'release_datetime'] = release_datetime
        except:
          df.loc[i, 'release_datetime'] = 'ERROR'
  
        print('request number ' + str(j) + ' - ' + str(df.loc[i, 'provider']) + ' - ERROR - ' + df.loc[i, 'url'])



  else:
    break
    
print('--- saving ---')
name = "News/news_scraped.csv"
df.to_csv(name, index=False)


Streaming output truncated to the last 5000 lines.
request number 1540 - James Picerno - SUCCESS - https://www.investing.com/analysis/us-stocks-treasury-tips-edged-up-in-last-weeks-global-selloff-200616505
request number 1541 - Brian Gilmartin - SUCCESS - https://www.investing.com/analysis/the-real-risk-to-growth-stock-and-style-investing-200616391
request number 1542 - AAATrade LTD - SUCCESS - https://www.investing.com/analysis/microsoft-stock-could-consolidate-200616313
request number 1543 - Tezcan Gecgil/Investing.com  - SUCCESS - https://www.investing.com/analysis/2-etfs-to-profit-from-supply-chain-uncertainties-200616224
request number 1544 - Zacks Investment Research - SUCCESS - https://www.investing.com/analysis/the-outlook-for-tech-stocks-in-a-rising-rate-environment-200616356
request number 1545 - Zacks Investment Research - SUCCESS - https://www.investing.com/analysis/value-growth-or-momentum-what-kind-of-investor-are-you-200616346
request number 1546 - Zacks Investment Resea

In [ ]:
t = pd.read_csv("News/news_scraped.csv")
t = t.drop_duplicates(['article_id','ticker','category'])

In [37]:
t

,ticker,title,category,content,release_date,provider,url,article_id,release_datetime
0,MSFT,Microsoft exposed 250M customer support records,news,Microsoft MSFT 0 1 discloses a database er...,2020-01-23,Seeking Alpha,https://invst.ly/pli4a,2065317.0,NaN
1,MSFT,Microsoft CEO Fears U S China Mistrust Will H...,news,Bloomberg Sign up here to receive the Dav...,2020-01-21,Bloomberg,https://www.investing.com/news/stock-market-ne...,2063837.0,NaN
2,MSFT,China Coronavirus Sells Off Asian Markets,opinion,Tuesday January 21 2020Market futures are in...,2020-01-20,Zacks Investment Research,https://www.investing.com/analysis/china-coron...,200500300.0,NaN
3,MSFT,Tech Daily Facebook Apple Digital Taxes More,opinion,Facebook s NASDAQ FB role in elections and...,2020-01-21,Zacks Investment Research,https://www.investing.com/analysis/tech-daily-...,200500369.0,NaN
4,MSFT,Chart Of The Day Can IBM s Results Help Avert...,opinion,International Business Machines NYSE IBM is ...,2020-01-21,Pinchas Cohen/Investing.com,https://www.investing.com/analysis/chart-of-th...,200500192.0,NaN
...,...,...,...,...,...,...,...,...,...
6521,GME,What To Expect When Trading Feels A Little Too...,opinion,Author's Publications The S&P 500 finished Wed...,2022-01-12,Jani Ziedins,https://www.investing.com/analysis/what-to-exp...,200614573.0,"Jan 12, 2022 11:44PM ET"
6522,GME,GameJustStop: Even An NFT Can't Help GameStop,opinion,"A year ago, the company that people could not ...",2022-01-08,Tim Knight,https://www.investing.com/analysis/gamejuststo...,200614068.0,"Jan 08, 2022 11:26PM ET"
6523,GME,The Smart Way To Trade This Week’s Selloff,opinion,Author's Publications The S&P 500 traded mostl...,2022-01-07,Jani Ziedins,https://www.investing.com/analysis/the-smart-w...,200613941.0,"Jan 07, 2022 01:27AM ET"
6524,GME,2021 Year In Review: Investing.com Comic Edition,opinion,Compiled by Jesse Cohen With just one trading ...,2022-01-03,Investing.com,https://www.investing.com/analysis/2021-year-i...,200613143.0,"Jan 03, 2022 01:35AM ET"


In [39]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "FB", "TSLA", "BRK-A", "TSM", "NVDA", "JPM", "GME"]
t = t[t.release_date<"2022-05-01"]
t = t[t.ticker.isin(tickers)]

t = t.reset_index(drop=True)

In [41]:
t.groupby(["ticker"])["release_date"].agg('min')

ticker
AAPL     2012-07-16
AMZN     2012-08-05
BRK-A    2011-11-08
FB       2012-05-18
GME      2012-05-17
GOOGL    2009-04-22
JPM      2008-11-05
MSFT     2008-10-15
NVDA     2009-08-07
TSLA     2012-05-23
TSM      2013-04-21
Name: release_date, dtype: object

In [42]:
t.groupby(["ticker"]).count()

,title,category,content,release_date,provider,url,article_id,release_datetime
ticker,,,,,,,,
AAPL,28799,28802,28802,28802,28802,28802,28802,8571
AMZN,14035,14036,14035,14036,14036,14036,14036,7706
BRK-A,413,413,391,413,413,413,413,413
FB,25958,25961,25958,25961,25951,25961,25961,25961
GME,1841,1845,1845,1845,1844,1845,1845,1845
GOOGL,11493,11494,11494,11494,11494,11494,11494,6323
JPM,6845,6846,6846,6846,6846,6846,6846,4246
MSFT,13766,13768,13768,13768,13768,13768,13768,5658
NVDA,2386,2386,2385,2386,2386,2386,2386,1971


In [ ]:
t[t.ticker=="AAPL"].head(10)

,ticker,title,category,content,release_date,provider,url,article_id,release_datetime
2959,AAPL,JPMorgan cautious ahead of Apple earnings,news,JPMorgan lifts its Apple AAPL 2 9 target f...,2020-01-28,Seeking Alpha,https://invst.ly/pnjv8,2068762.0,NaN
2960,AAPL,FAANG s Fall but Get Some Wall Street Love,news,By Kim Khan \nInvesting com The FAANG stocks...,2020-01-28,Investing.com,https://www.investing.com/news/stock-market-ne...,2068765.0,NaN
2961,AAPL,Wall Street tumbles as virus fuels economic worry,news,By Chuck Mikolajczak NEW YORK Reuters U S ...,2020-01-28,Reuters,https://www.investing.com/news/stock-market-ne...,2068311.0,NaN
2962,AAPL,Earnings Watch Apple and AMD to take earnings...,news,Two of the best performing tech stocks of 2019...,2020-01-28,MarketWatch,https://invst.ly/pnlbs,2068906.0,NaN
2963,AAPL,Day Ahead Top 3 Things to Watch for Jan 28,news,By Yasin Ebrahim and Kim Khan \n1 Apple Readi...,2020-01-28,Investing.com,https://www.investing.com/news/stock-market-ne...,2068907.0,NaN
2964,AAPL,China and Apple s TV service will be under the...,news,Apple is set to report fiscal first quarter re...,2020-01-28,CNBC,https://invst.ly/pnlj4,2068914.0,NaN
2965,AAPL,Stocks Europe to Edge Higher Virus Prompts ...,news,By Peter Nurse \nInvesting com European stoc...,2020-01-28,Investing.com,https://www.investing.com/news/stock-market-ne...,2069039.0,NaN
2966,AAPL,Tech Earnings Expected To Turn Around,opinion,Total earnings for the 87 S P 500 members that...,2020-01-28,Zacks Investment Research,https://www.investing.com/analysis/tech-earnin...,200502045.0,NaN
2967,AAPL,Coronavirus outbreak may disrupt Apple s iPhon...,news,Reuters Apple Inc s O AAPL plan to ramp ...,2020-01-28,Reuters,https://www.investing.com/news/technology-news...,2069040.0,NaN
2968,AAPL,These Tech Bigwigs Are Set To Beat Earnings Th...,opinion,With Tech stealing the show in the last earnin...,2020-01-28,Zacks Investment Research,https://www.investing.com/analysis/these-tech-...,200501712.0,NaN
